# GFAN Seizure Detection Pipeline

Main execution notebook for the GFAN seizure detection pipeline. This notebook orchestrates data preprocessing, feature extraction, model training, and evaluation using Leave-One-Subject-Out cross-validation.

### 1. Imports

In [1]:
import subprocess
import sys

# Install required packages
packages = ["mne", "pyedflib"]
for package in packages:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"Successfully installed {package}")
    except subprocess.CalledProcessError as e:
        print(f"Failed to install {package}: {e}")

print("Package installation completed!")

Package installation completed!
Package installation completed!


In [2]:
import os
import glob
import json
import numpy as np
import torch
import pandas as pd
from tqdm import tqdm  # Using regular tqdm instead of notebook version

from src.data_preprocessing import CHBMITDataProcessor, load_chb_mit_annotations
from src.spectral_decomposition import MultiScaleSTFT
from src.graph_construction import create_graph_from_windows
from src.training import EEGDataset, LeaveOneSubjectOutValidator
from src.gfan_model import GFAN

### 2. Configuration

In [ ]:
config = {
    'data': {
        'path': 'chb-mit-scalp-eeg-database-1.0.0',
        'target_fs': 256,
        'window_size': 2.0,
        'overlap': 0.5,
        'n_subjects_to_process': None  # Process ALL subjects with A6000's 48GB VRAM
    },
    'features': {
        'window_sizes': [1.0, 2.0, 4.0],
        'hop_ratio': 0.25,
        'log_transform': True
    },
    'graph': {
        'method': 'hybrid',
        'spatial_weight': 0.5,
        'functional_weight': 0.5
    },
    'model': {
        'n_channels': 18, # Will be updated based on data
        'spectral_features_dims': [129, 257, 513], # Will be updated
        'hidden_dims': [256, 128, 64],  # Increased capacity for A6000
        'n_classes': 2,
        'sparsity_reg': 0.01,
        'dropout_rate': 0.2,
        'uncertainty_method': 'mc_dropout',
        'fusion_method': 'attention'
    },
    'trainer': {
        'learning_rate': 1e-4,
        'weight_decay': 1e-5,
        'class_weights': [1.0, 10.0], # Will be updated based on data
        'sparsity_weight': 0.01,
        'kl_weight': 1e-6,
        'epochs': 100,  # Increased for full dataset training
        'batch_size': 128,  # Larger batch size for A6000
        'gradient_accumulation_steps': 4  # Effective batch size: 512
    },
    'validation': {
        'n_folds': None  # Full LOSO validation for complete evaluation
    },
    'results_path': 'results/final_run_summary.json'
}

### 3. Device Setup

In [ ]:
# Configure device for NVIDIA A6000 GPU environment
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using CUDA GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    # Enable optimizations for A6000
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
else:
    device = torch.device("cpu")
    print("CUDA not available. Using CPU.")
config['trainer']['device'] = device

Using Apple Metal (MPS) for acceleration.


### 4. Initialization

In [6]:
processor = CHBMITDataProcessor(
    target_fs=config['data']['target_fs'],
    window_size=config['data']['window_size'],
    overlap=config['data']['overlap']
)
stft_extractor = MultiScaleSTFT(
    fs=config['data']['target_fs'],
    window_sizes=config['features']['window_sizes'],
    hop_ratio=config['features']['hop_ratio']
)

### 5. Data Loading and Preprocessing

In [ ]:
import gc  # For garbage collection

# Initialize storage
all_windows, all_labels, all_subjects = [], [], []
all_spectral_features = [[] for _ in config['features']['window_sizes']]

subject_dirs = sorted([d for d in glob.glob(os.path.join(config['data']['path'], 'chb*')) if os.path.isdir(d)])
# Process ALL subjects - no restrictions for A6000 environment
print(f"Processing ALL {len(subject_dirs)} subjects with full dataset...")

# Process each subject with optimized memory management for A6000
for subject_idx, subject_dir in enumerate(subject_dirs):
    print(f"\nProcessing Subject {subject_idx + 1}/{len(subject_dirs)}: {os.path.basename(subject_dir)}")
    
    subject_id = int(os.path.basename(subject_dir).replace('chb', ''))
    summary_file = os.path.join(subject_dir, f"{os.path.basename(subject_dir)}-summary.txt")
    
    try:
        annotations = load_chb_mit_annotations(summary_file)
        print(f"  Loaded annotations for {len(annotations)} files")
    except Exception as e:
        print(f"  Warning: Could not load annotations for {subject_dir}. Error: {e}")
        continue
    
    edf_files = sorted(glob.glob(os.path.join(subject_dir, '*.edf')))
    print(f"  Found {len(edf_files)} EDF files")
    
    # Process ALL files per subject - no memory restrictions
    subject_windows = []
    subject_labels = []
    subject_spectral_features = [[] for _ in config['features']['window_sizes']]
    
    files_processed = 0
    for edf_file in tqdm(edf_files, desc=f"Processing files for subject {subject_id}", leave=False):
        file_name = os.path.basename(edf_file)
        seizure_info = annotations.get(file_name, [])
        
        try:
            windows, labels, channels = processor.process_file(edf_file, seizure_info)
            if windows is None or len(windows) == 0:
                print(f"    No windows extracted from {file_name}")
                continue
            
            print(f"    Extracted {len(windows)} windows from {file_name}, seizure events: {len(seizure_info)}")
            
            # No window limits - process ALL windows with A6000's 48GB VRAM
            print(f"    Processing all {len(windows)} windows (no memory restrictions)")
            
            # Extract spectral features in optimized batches for A6000
            file_spectral_features = [[] for _ in config['features']['window_sizes']]
            batch_size = 64  # Larger batch size for A6000's memory capacity
            
            for batch_start in range(0, len(windows), batch_size):
                batch_end = min(batch_start + batch_size, len(windows))
                
                for i in range(batch_start, batch_end):
                    try:
                        multiscale_stft = stft_extractor.compute_multiscale_stft(windows[i])
                        for scale_idx, stft_result in enumerate(multiscale_stft):
                            file_spectral_features[scale_idx].append(stft_result['magnitude'])
                    except Exception as e:
                        print(f"      Warning: STFT failed for window {i}: {e}")
                        continue
                
                # Less frequent cleanup for better performance
                if batch_start % (batch_size * 10) == 0:
                    gc.collect()
            
            # Add all processed data
            if any(len(features) > 0 for features in file_spectral_features):
                min_features = min(len(features) for features in file_spectral_features if len(features) > 0)
                if min_features > 0:
                    subject_windows.append(windows[:min_features])
                    subject_labels.append(labels[:min_features])
                    
                    for scale_idx in range(len(config['features']['window_sizes'])):
                        if len(file_spectral_features[scale_idx]) >= min_features:
                            subject_spectral_features[scale_idx].extend(file_spectral_features[scale_idx][:min_features])
                
                files_processed += 1
            
        except Exception as e:
            print(f"    Warning: Could not process file {edf_file}. Error: {e}")
            continue
    
    # Add subject data to global collections
    if len(subject_windows) > 0:
        subject_windows_concat = np.concatenate(subject_windows, axis=0)
        subject_labels_concat = np.concatenate(subject_labels, axis=0)
        
        all_windows.append(subject_windows_concat)
        all_labels.append(subject_labels_concat)
        all_subjects.extend([subject_id] * len(subject_windows_concat))
        
        for scale_idx in range(len(config['features']['window_sizes'])):
            if len(subject_spectral_features[scale_idx]) > 0:
                all_spectral_features[scale_idx].append(np.array(subject_spectral_features[scale_idx]))
    
    print(f"  Successfully processed {files_processed}/{len(edf_files)} files for subject {subject_id}")
    print(f"  Subject {subject_id} total windows: {len(subject_windows_concat) if len(subject_windows) > 0 else 0}")
    
    # Periodic cleanup every 5 subjects
    if (subject_idx + 1) % 5 == 0:
        gc.collect()
        print(f"  Memory cleanup after subject {subject_idx + 1}")

# Check if we have any data before final concatenation
if len(all_windows) == 0:
    raise ValueError("No data was successfully processed! Check dataset path and file permissions.")

print("Final concatenation of all processed data...")

# Concatenate with error checking
try:
    final_windows = np.concatenate(all_windows, axis=0)
    final_labels = np.concatenate(all_labels, axis=0)
    final_subjects = np.array(all_subjects)
    
    print(f"Windows shape: {final_windows.shape}")
    print(f"Labels shape: {final_labels.shape}")
    print(f"Subjects: {len(final_subjects)} total")
    
except Exception as e:
    print(f"Error during concatenation: {e}")
    raise ValueError("Failed to concatenate data")

# Handle spectral features concatenation
final_spectral_features = []
for scale_idx in range(len(config['features']['window_sizes'])):
    if len(all_spectral_features[scale_idx]) > 0:
        try:
            concatenated = np.concatenate(all_spectral_features[scale_idx], axis=0)
            final_spectral_features.append(concatenated)
            print(f"Scale {scale_idx} spectral features shape: {concatenated.shape}")
        except Exception as e:
            print(f"Error concatenating spectral features for scale {scale_idx}: {e}")
            raise
    else:
        raise ValueError(f"No spectral features found for scale {scale_idx}")

print(f"\nFinal data processing summary:")
print(f"Total windows: {len(final_windows)}")
print(f"Total data size: {final_windows.nbytes / 1024**3:.2f} GB")
print(f"Class distribution: {pd.Series(final_labels).value_counts().to_dict()}")
print(f"Unique subjects: {len(np.unique(final_subjects))}")
print(f"Spectral features shapes: {[f.shape for f in final_spectral_features]}")
print(f"Total spectral features size: {sum(f.nbytes for f in final_spectral_features) / 1024**3:.2f} GB")

# Final cleanup
del all_windows, all_labels, all_spectral_features
gc.collect()
print("Full dataset processing completed successfully!")

Starting data processing for 3 subjects...

Processing Subject 1/3: chb01
  Loaded annotations for 42 files
  Found 42 EDF files


Processing files for subject 1:   0%|          | 0/2 [00:00<?, ?it/s]

Starting data processing for 3 subjects...

Processing Subject 1/3: chb01
  Loaded annotations for 42 files
  Found 42 EDF files


Processing files for subject 1:   0%|          | 0/2 [00:00<?, ?it/s]

    Extracted 3598 windows from chb01_01.edf, seizure events: 0
    Limited to 50 windows for memory safety


...

Processing files for subject 3:   0%|          | 0/2 [00:00<?, ?it/s]

    Extracted 3598 windows from chb03_01.edf, seizure events: 0
    Limited to 50 windows for memory safety


Processing files for subject 3:  50%|█████     | 1/2 [00:07<00:07,  7.19s/it]

    Extracted 3598 windows from chb03_02.edf, seizure events: 0
    Limited to 50 windows for memory safety


  Successfully processed 2/2 files for subject 3
Final concatenation of all processed data...
Windows shape: (300, 23, 512)
Labels shape: (300,)
Subjects: 300 total
Scale 0 spectral features shape: (300, 23, 129, 9)
Scale 1 spectral features shape: (300, 23, 257, 5)
Scale 2 spectral features shape: (300, 23, 257, 5)

Data processing summary:
Total windows: 300
Class distribution: {0.0: 300}
Unique subjects: 3
Spectral features shapes: [(300, 23, 129, 9), (300, 23, 257, 5), (300, 23, 257, 5)]
Data processing completed successfully!


In [ ]:
# Reload the spectral decomposition module
import importlib
import src.spectral_decomposition
importlib.reload(src.spectral_decomposition)
from src.spectral_decomposition import MultiScaleSTFT

# Recreate the STFT extractor with the fixed code
stft_extractor = MultiScaleSTFT(
    fs=config['data']['target_fs'],
    window_sizes=config['features']['window_sizes'],
    hop_ratio=config['features']['hop_ratio']
)

print("Module reloaded and extractor recreated.")

In [52]:
# Reset variables for clean data processing
all_windows, all_labels, all_subjects = [], [], []
all_spectral_features = [[] for _ in config['features']['window_sizes']]
print("Variables reset for clean processing run.")

Variables reset for clean processing run.


### 6. Graph Construction

In [28]:
print("Constructing graph...")
graph_info = create_graph_from_windows(
    final_windows, 
    channels, 
    method=config['graph']['method']
)
# Move graph tensors to the correct device
for key in ['adjacency', 'laplacian', 'eigenvalues', 'eigenvectors']:
    graph_info[key] = graph_info[key].to(device)
print("Graph constructed and moved to device.")

Constructing graph...
Graph constructed and moved to device.


### 7. Update Config and Create Dataset

In [29]:
config['model']['n_channels'] = final_windows.shape[1]
config['model']['spectral_features_dims'] = [f.shape[1] for f in final_spectral_features]

# Update class weights based on data imbalance
class_counts = pd.Series(final_labels).value_counts()
if 1 in class_counts and 0 in class_counts:
    weight_for_class_0 = len(final_labels) / (2 * class_counts[0])
    weight_for_class_1 = len(final_labels) / (2 * class_counts[1])
    config['trainer']['class_weights'] = [weight_for_class_0, weight_for_class_1]
else:
    # Handle case where one class is missing in the processed subset
    config['trainer']['class_weights'] = [1.0, 1.0]

print(f"Calculated class weights: {config['trainer']['class_weights']}")

# Create the full dataset
full_dataset = EEGDataset(
    windows=final_windows,
    labels=final_labels,
    spectral_features=final_spectral_features,
    subjects=final_subjects,
    training=False # Augmentation is handled inside the trainer/validator
)

Calculated class weights: [1.0, 1.0]


### 8. Run Leave-One-Subject-Out Validation

In [34]:
print("Starting Leave-One-Subject-Out cross-validation...")
validator = LeaveOneSubjectOutValidator(
    model_config=config['model'],
    trainer_config=config['trainer']
)

results = validator.validate(
    dataset=full_dataset,
    graph_info=graph_info,
    n_folds=config['validation']['n_folds']
)

Starting Leave-One-Subject-Out cross-validation...

----- Fold 1/3 -----


Training:   0%|          | 0/5 [00:00<?, ?it/s]



Starting Leave-One-Subject-Out cross-validation...

----- Fold 1/3 -----


Training:   0%|          | 0/5 [00:00<?, ?it/s]



TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.

### 9. Save Final Results

In [35]:
print("Validation finished. Saving results...")
os.makedirs(os.path.dirname(config['results_path']), exist_ok=True)
validator.save_results(config['results_path'])

print(f"Results saved to {config['results_path']}")
print("\n--- Summary Metrics ---")
summary = validator.get_summary_metrics()
if summary:
    for key, value in summary.items():
        print(f"{key}: {value:.4f}")

Starting memory-efficient data processing...
Using temporary directory: /var/folders/q1/0h_x7yc145l73wy2vgm5fd300000gn/T/tmpywq4u_vw
Processing 3 subjects...


Starting memory-efficient data processing...
Using temporary directory: /var/folders/q1/0h_x7yc145l73wy2vgm5fd300000gn/T/tmpywq4u_vw
Processing 3 subjects...


Processing Subjects:   0%|          | 0/3 [00:00<?, ?it/s]

Starting memory-efficient data processing...
Using temporary directory: /var/folders/q1/0h_x7yc145l73wy2vgm5fd300000gn/T/tmpywq4u_vw
Processing 3 subjects...


Processing Subjects:   0%|          | 0/3 [00:00<?, ?it/s]

...

  Subject 1: 5000 total windows saved to temp file

Processing subject 2/3: chb02
  Processing file 1/10: chb02_01.edf
    Extracted 500 windows
    Extracted 500 windows
    Successfully processed 500 windows
  Processing file 2/10: chb02_02.edf
    Successfully processed 500 windows
  Processing file 2/10: chb02_02.edf
    Extracted 500 windows
    Extracted 500 windows
    Successfully processed 500 windows
  Processing file 3/10: chb02_03.edf
    Successfully processed 500 windows
  Processing file 3/10: chb02_03.edf
    Extracted 500 windows
    Extracted 500 windows
    Successfully processed 500 windows
  Processing file 4/10: chb02_04.edf
    Successfully processed 500 windows
  Processing file 4/10: chb02_04.edf
    Extracted 500 windows
    Extracted 500 windows
    Successfully processed 500 windows
  Processing file 5/10: chb02_05.edf
    Successfully processed 500 windows
  Processing file 5/10: chb02_05.edf
    Extracted 500 windows
    Extracted 500 windows
    Successfu

Processing Subjects:  67%|██████▋   | 2/3 [03:42<01:51, 111.61s/it]

  Subject 2: 5000 total windows saved to temp file

Processing subject 3/3: chb03
  Processing file 1/10: chb03_01.edf
    Extracted 500 windows
    Extracted 500 windows
    Successfully processed 500 windows
  Processing file 2/10: chb03_02.edf
    Successfully processed 500 windows
  Processing file 2/10: chb03_02.edf
    Extracted 500 windows
    Extracted 500 windows
    Successfully processed 500 windows
  Processing file 3/10: chb03_03.edf
    Successfully processed 500 windows
  Processing file 3/10: chb03_03.edf
    Extracted 500 windows
    Extracted 500 windows
    Successfully processed 500 windows
  Processing file 4/10: chb03_04.edf
    Successfully processed 500 windows
  Processing file 4/10: chb03_04.edf
    Extracted 500 windows
    Extracted 500 windows
    Successfully processed 500 windows
  Processing file 5/10: chb03_05.edf
    Successfully processed 500 windows
  Processing file 5/10: chb03_05.edf
    Extracted 500 windows
    Extracted 500 windows
    Successfu

Processing Subjects: 100%|██████████| 3/3 [05:33<00:00, 111.27s/it]

Error processing subject chb-mit-scalp-eeg-database-1.0.0/chb03: [Errno 28] No space left on device

Data processing summary:
- Total files processed: 30
- Total windows processed: 15000
- Subjects with data: 2
- Temporary files created in: /var/folders/q1/0h_x7yc145l73wy2vgm5fd300000gn/T/tmpywq4u_vw
SUCCESS: Data processing completed. Temporary files created for memory efficiency.
Next step: Load and combine data as needed for training.


In [ ]:
# A6000 GPU Optimization Settings
print("=== A6000 GPU Optimization Settings ===")

# Enable optimizations for maximum performance
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# Set memory management
torch.cuda.empty_cache()
print(f"Initial GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"GPU memory reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
print(f"Max GPU memory: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")

# Display final memory usage
total_params = sum(p.numel() for p in validator.model.parameters())
trainable_params = sum(p.numel() for p in validator.model.parameters() if p.requires_grad)
print(f"Model parameters: {total_params:,} total, {trainable_params:,} trainable")
print(f"Model size: ~{total_params * 4 / 1024**2:.1f} MB (FP32)")

# Show dataset statistics
print(f"\n=== Dataset Statistics ===")
print(f"Total samples processed: {len(final_windows):,}")
print(f"Memory usage - Windows: {final_windows.nbytes / 1024**3:.2f} GB")
print(f"Memory usage - Features: {sum(f.nbytes for f in final_spectral_features) / 1024**3:.2f} GB")
print(f"Total dataset memory: {(final_windows.nbytes + sum(f.nbytes for f in final_spectral_features)) / 1024**3:.2f} GB")

# Estimated training time
estimated_time_per_epoch = len(final_windows) / (config['trainer']['batch_size'] * 60)  # minutes
print(f"Estimated time per epoch: {estimated_time_per_epoch:.1f} minutes")
print(f"Total estimated training time: {estimated_time_per_epoch * config['trainer']['epochs'] / 60:.1f} hours")